1.파이썬-> 구글스프레드시트로 연결시켜서 옮기기

2.사이트에서 크롤링

3.조건 1.OWNER / 2.has image / 3. Price 5,000이하

In [164]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]

#json파일
json_file_name = 'sunlit-cove-371001-e6ac09769edd.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#URL가져오기
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/10lPp04ItUu7w0Lgt3gRXD_rvtgW8ZLtbfjRcwnmnkyI/edit?pli=1#gid=0'
doc = gc.open_by_url(spreadsheet_url)

Page_i = 1
sheet_name = "page " + str(Page_i)
ws = doc.add_worksheet(title=sheet_name, rows=150, cols=26)
worksheet = doc.worksheet(sheet_name) # 시트선택
worksheet.update_acell('A1', 'DATE')
worksheet.update_acell('B1', 'LOCATION')
worksheet.update_acell('C1', 'TITLE')
worksheet.update_acell('D'+str(1), 'IMAGE')

{'spreadsheetId': '10lPp04ItUu7w0Lgt3gRXD_rvtgW8ZLtbfjRcwnmnkyI',
 'updatedRange': "'page 1'!D1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [3]:
#마무리
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests

#Gspread 연동
scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]

#json파일
json_file_name = 'sunlit-cove-371001-e6ac09769edd.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#URL가져오기
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/10lPp04ItUu7w0Lgt3gRXD_rvtgW8ZLtbfjRcwnmnkyI/edit?pli=1#gid=0'
doc = gc.open_by_url(spreadsheet_url)


Page_i = 1
sheet_name = "page " + str(Page_i)

ws = doc.add_worksheet(title=sheet_name, rows=150, cols=26)#시트생성

worksheet = doc.worksheet(sheet_name)#시트선택
worksheet.update_acell('A1', 'DATE')
worksheet.update_acell('B1', 'LOCATION')
worksheet.update_acell('C1', 'TITLE')
worksheet.update_acell('D1', 'IMAGE')

driver = webdriver.Chrome()
driver.get('https://sfbay.craigslist.org/search/cta?hasPic=1&max_price=5000&purveyor=owner#search=1~gallery~0~0')
sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


#차량이름, 위치, 날짜
title = soup.select('ol li.cl-search-result div.title-blob')# #은 아이디, 띄어쓰기 하위 속석 .은 class
created_date_and_location = soup.select('ol div.meta')
title_size = len(title)
for i in range(title_size):
    title_text = title[i].text
    split = created_date_and_location[i].text.split('·')
    created_date = split[0]
    location = split[1]
    first_link = soup.select('ol div a.main')
    first_link_size = len(first_link)
    #print('created_date:',created_date,'location:',location,'title:',title_text)
    print(i+2)
    print('title_text:',title_text)
    print('location:',location)
    print('date:',created_date)
    print()



2
title_text 2001 Ford Taurus SES
location: hayward / castro valley
date: 12/14

3
title_text San rail buggy
location: atherton
date: 12/14

4
title_text 2003, jeep liberty sports, clean title,current reg, miles,240,k
location: Hercules
date: 12/14

5
title_text 2003 Buick La Sabre, Low Miles
location: santa cruz
date: 12/14

6
title_text Super clean well maintained 104k miles 2007 Hyundai accent Gls
location: San Francisco
date: 12/14

7
title_text 1993 Cadillac STS 80k miles 2nd owner & smogged
location: santa rosa
date: 12/14

8
title_text 1983 Honda Accord LX
location: santa clara
date: 12/14

9
title_text Dodge Durango
location: vallejo / benicia
date: 12/14

10
title_text 2006 bmw 325i
location: fremont / union city / newark
date: 12/14

11
title_text 2000 civic
location: San Francisco
date: 12/14

12
title_text 2011 Chevy hhr 129,000 miles
location: santa rosa
date: 12/14

13
title_text 1999 Ford F150
location: San Jose
date: 12/14

14
title_text 2003 Honda Odyssey
location: San

104
title_text 1991 Mercedes Benz 300CE 2-Door Coupe (C124)
location: richmond / point / annex
date: 12/13

105
title_text 2007 Chevy Suburban LTZ
location: mountain view
date: 12/13

106
title_text Ford Crown Victoria Ltd Wagon
location: san carlos
date: 12/13

107
title_text Cadillac dts
location: concord / pleasant hill / martinez
date: 12/13

108
title_text 2013 toyot tacoma 4x4 Limited
location: Penngrove
date: 12/13

109
title_text Land Rover Freelander 2003
location: gilroy
date: 12/13

110
title_text 1998 JEEP GRAND CHEROKEE LIMITED 4X4. Running good $2400 obo
location: pittsburg / antioch
date: 12/13

111
title_text Ford Taurus
location: south bay area
date: 12/13

112
title_text 2008 Prius
location: redwood city
date: 12/13

113
title_text 2010 Nissan rogue very clean
location: hayward / castro valley
date: 12/13

114
title_text 2000 Mitsubishi fuso box truck needs work
location: San Jose
date: 12/13

115
title_text 1940 Chrysler Royal
location: brentwood / oakley
date: 12/13

In [ ]:
    try : 
        worksheet.update_acell('A'+str(i+2), created_date)
    except:
        print('예외가 발생했습니다1상.')
        row_data = worksheet.row_values(1)
        sleep(10)
        row_data = worksheet.row_values(1)
        print(row_data)
        worksheet.update_acell('A'+str(i+2), created_date)
        print('예외가 발생했습니다1하.')
    try : 
        worksheet.update_acell('B'+str(i+2), location)
    except Exception as e:
        print(e)
        print('예외가 발생했습니다2상.')
        sleep(10)
        worksheet.update_acell('B'+str(i+2), location)
        print('예외가 발생했습니다2하.')
    try : 
        worksheet.update_acell('C'+str(i+2), title_text)
    except:
        print('예외가 발생했습니다3상.')
        sleep(10)
        worksheet.update_acell('C'+str(i+2), title_text)
        print('예외가 발생했습니다3하.')

In [ ]:
    try :
        for i in range(first_link_size):
            print((first_link[i])['href'])
            res = requests.get((first_link[i])['href'])
            soup2 = BeautifulSoup(res.content, 'html.parser')
    except:
        print('예외발생')

In [115]:
#해당 게시글에서 사진링크
import requests
from bs4 import BeautifulSoup

res = requests.get('https://sfbay.craigslist.org/pen/cto/7568245359.html')
soup2 = BeautifulSoup(res.content, 'html.parser')
#print(soup2)
soup2_select = soup2.select('#thumbs a.thumb')
for i in range (len(soup2_select)):
    print((soup2_select[i])['href'])

https://images.craigslist.org/00e0e_95QXM0gpitUz_0t20CI_600x450.jpg
https://images.craigslist.org/00303_5knd96q8vIsz_0CI0t2_600x450.jpg
https://images.craigslist.org/00t0t_MvDCbbR3DQz_0lM0t2_600x450.jpg
https://images.craigslist.org/00O0O_5WYmWw73Dbzz_0t20CI_600x450.jpg
https://images.craigslist.org/00S0S_7GKbXCATtQez_0t20CI_600x450.jpg
https://images.craigslist.org/00q0q_gDjxlfqg255z_0t20CI_600x450.jpg
https://images.craigslist.org/00a0a_2IRU7Y6GxDmz_0CI0t2_600x450.jpg


In [9]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep


driver = webdriver.Chrome()
driver.get('https://orangecounty.craigslist.org/search/cta#search=1~gallery~0~0')
sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)
#elements = driver.find_elements_by_css_selector('#search-results-page-1 > ol')
#보통 html만 사용하여 페이지를 다 표현하지 않고, css로 동적인 이미지 표현한 것도 있어서 
#단순히 html 파싱으로는 가져오는게 한계가 있다.


<html><head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="craigslist" property="og:site_name"/>
<meta content="preview" name="twitter:card"/>
<meta content="orange co cars &amp; trucks - craigslist" property="og:title"/>
<meta content="orange co cars &amp; trucks - craigslist" name="description"/>
<meta content="orange co cars &amp; trucks - craigslist" property="og:description"/>
<meta content="https://orangecounty.craigslist.org/search/cta" property="og:url"/>
<title>orange co cars &amp; trucks - craigslist</title>
<link href="https://orangecounty.craigslist.org/search/cta" rel="canonical"/>
<style type="text/css">
        body {
            font-family: sans-serif;
        }

        #no-js {
            position: fixed;
            z-index: 100000;
            top: 0;
            bottom: 0;
            left: 0;
            right: 0;
            overflow:auto;
      

In [69]:
import requests
from bs4 import BeautifulSoup as bs
import json
url = 'https://sapi.craigslist.org/web/v7/postings/search/full?batch=103-0-360-0-0&cc=US&lang=en&searchPath=cta'

response = requests.get(url)
bsObj = bs(response.content,'html.parser')
jsonObj = response.json()
#print(bsObj)
#print(jsonObj)
json_formatted_str = json.dumps(jsonObj, indent=2)

print(json_formatted_str)
#print(jsonObj["items"])

{
  "apiVersion": 7,
  "configId": "e56cb584be3b1cbc1834e32baa9195023f7557765c85bf463687c991f28721d5",
  "configTs": 1670966748,
  "data": {
    "areas": {
      "103": {
        "areaUnits": "ft",
        "currency": "USD",
        "distanceUnits": "mi",
        "name": "orangecounty"
      }
    },
    "cacheTs": 1670970179,
    "canonicalUrl": "//orangecounty.craigslist.org/search/cta",
    "categoryAbbr": "cta",
    "decode": {
      "locationDescriptions": [
        0,
        "Costa Mesa",
        "Santa Ana",
        "+ Best Deal Auto Brokers",
        "TOP-DOLLAR FOR TRADES",
        "Orange",
        "Huntington Beach/Westminister",
        "orange",
        "Irvine",
        "SANTA ANA",
        "Anaheim hills",
        "Temecula",
        "Anaheim",
        "Tonight ee tacos for bidders -18442 Vanburen. Blvd.Riverside",
        "Free tacos for all bidders -18442 Vanburen. Blvd.Riverside",
        "- as low as $499 Down oac* All Credit Accepted!!!",
        "Anaheim/Fullerton

In [59]:
print(jsonObj["data"]["decode"]["locationDescriptions"])

[0, 'Costa Mesa', 'Anaheim', '+ Best Deal Auto Brokers', 'Est. payment OAC', '** FAST APPROVALS! SE HABLA ESPANOL! **', 'Corona', 'Garden grove', 'FAM Vans Inc', 'Westminster', 'Garden. Grove', 'Temecula', '- as low as $499 Down oac* All Credit Accepted!!!', 'Newport Beach, CA', 'GARDENA', 'Yorba Linda', 'Tustin', '_Ford_ _Mustang_ _Coupe_', 'Huntington Beach', '_Chevrolet_ _Silverado 1500_ _Truck_', 'Irvine', '2018 Jeep Wrangler Unlimited Sport S 4x4', '2021 Lexus NX NX 300 F SPORT FWD', '2019 Infiniti QX50 ESSENTIAL FWD', '2019 Lexus ES ES 350 FWD', '_Toyota_ _TUNDRA 4WD_ _Truck_', '2019 Hyundai Tucson SE FWD', '2016 Honda Civic Sedan 4dr CVT LX', '2015 Hyundai Sonata 4dr Sdn 2.4L Limited', '2015 Infiniti Q40 4dr Sdn RWD', 'Great Work Truck LIFTGATE Tow Package Service Bed', 'Garden Grove', '_Ram_ _3500_ _Truck_', 'CLEAN TITLE, CURRENT REGISTRATION, CALL ME 562 3827323', '822 W. VALLEY BLVD.BLOOMINGTON CA 92316', 'Capistrano', 'Fullerton', 'Buena park', '26705 Ynez Rd Temecula, CA 92

In [85]:
import requests
from bs4 import BeautifulSoup as bs
import json
url = 'https://sapi.craigslist.org/web/v7/postings/search/full?batch=103-0-360-0-0&cc=US&hasPic=1&lang=en&max_price=5000&purveyor=owner&searchPath=cta'

response = requests.get(url)
print(response)
jsonObj = response.json()
#print(response.headers)
header = {'Last-Modified': 'Tue'}
#print(bsObj)
#print(jsonObj)
response = requests.get(url, headers = header)
jsonObj = response.json()
#print(jsonObj)
json_formatted_str = json.dumps(jsonObj, indent=2)
#print(json_formatted_str)
print(jsonObj['data']['items'])
#print(jsonObj["items"])

<Response [200]>
[[5978174, 1383222, 145, 2950, '1:1~34.1144~-117.6583', [4, '3:00G0G_3uCj7tgPKtqz_0CI0t2', '3:00303_fmdmKJsDfqRz_0CI0t2', '3:00S0S_9O9SHL8XM3oz_0CI0t2', '3:00101_4ZxWPluiffAz_0CI0t2', '3:00o0o_2xaOXdoVsiJz_0CI0t2', '3:01111_c3Qrub8cXnLz_0CI0t2', '3:01313_YNSzXsNg2Mz_0CI0t2', '3:00707_2oCuUHJtahjz_0CI0t2'], '98 Toyota Tacoma'], [5948272, 1378713, 145, 3400, '1:2~33.6654~-117.9686', [4, '3:00r0r_fsuNVM9wPc9z_0CI0t2', '3:00000_cRA48APNURhz_0CI0t2', '3:00O0O_36gKsZTdDw1z_0CI0t2', '3:00N0N_1b8PDYwzzOIz_0CI0t2', '3:00d0d_2LVF4hrpEmYz_0CI0t2'], '02 Ford e350 super duty extended cargo'], [5945775, 1378352, 145, 4750, '1:3~33.4984~-117.7064', [4, '3:00L0L_69r09CXa9AWz_0CI0t2', '3:00q0q_2dMlEjqs3x1z_0t20CI', '3:00303_cHJD9mLsOSaz_0CI0t2', '3:00000_3esT4EJQYIgz_0t20CI', '3:00P0P_7MHjaQD6UlAz_0CI0t2', '3:00q0q_1OkOEhKuWybz_0t20CI', '3:00A0A_2Z2FoJ70WHsz_0CI0t2', '3:00o0o_lJ25zrumRnmz_0CI0t2', '3:00F0F_1j97W08fOzBz_0CI0t2'], '2009 Toyota matrix'], [5936786, 1377304, 145, 4750, '1:4